## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

I_lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)     

def fourier(img):
    F_img = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    F_img = np.fft.fftshift(F_img, [0, 1])
    Amplitude, phase = cv2.cartToPolar(F_img[:,:,0], F_img[:,:,1])

    AmplitudeLog = np.log10(Amplitude + 1)

    return F_img, AmplitudeLog, phase

def ploting_function(img1,img2,img3):
    f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(15,5))
    ax1.imshow(img1, 'gray')
    ax1.set_title('Oryginał')
    ax1.axis('off')
    ax2.imshow(img2, 'gray')
    ax2.set_title('Amplitude')
    ax2.axis('off')
    ax3.imshow(img3, 'gray')
    ax3.set_title('Phase')
    ax3.axis('off')

def fourier2(img):
    FRow = np.fft.fft(img, axis=0)
    F_img = np.fft.fft(FRow, axis=1)

    Amplitude, phase = cv2.cartToPolar(F_img.real, F_img.imag)
    AmplitudeLog = np.log10(Amplitude + 1)

    return F_img, AmplitudeLog, phase

def inverse_fourier(img):
    img_shifted = np.fft.ifftshift(img, [0, 1])
    img_idft = cv2.idft(img_shifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    imgIFFT = cv2.magnitude(img_idft[:,:,0], img_idft[:,:,1])
    img_inv = np.round(imgIFFT).astype('uint8')

    return img_inv

size = 21
hanning_window = np.hanning(size)
hanning_window_2d = np.outer(hanning_window, hanning_window)

F_fcn_lena, magnitude_lena, phase_lena = fourier(I_lena)

window_size=[size, size]

FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(window_size[0]))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, window_size[1]]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(window_size[1]))
FSpaceColsM = np.outer(np.ones([1, window_size[0]]), FSpaceCols)

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
FilterF = FreqR >= 0.2

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

Fiiltr_res = hanning_window_2d * FilterFI

Filtr_zp = cv2.copyMakeBorder(Fiiltr_res, 246, 245, 246, 245, cv2.BORDER_CONSTANT, value=0).astype('float32')
F_fcn_filtr = cv2.dft(Filtr_zp, flags=cv2.DFT_COMPLEX_OUTPUT)
F_fcn_filtr = np.fft.fftshift(F_fcn_filtr, [0,1])

F_fcn_filtr_abs = np.abs(F_fcn_filtr)

lenaSize = I_lena.shape
FSpaceRows_512 = 2*np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
FSpaceRowsM_512 = np.outer(FSpaceRows_512, np.ones([1, lenaSize[1]]))
FSpaceCols_512 = 2*np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
FSpaceColsM_512 = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols_512)  

figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM_512, FSpaceColsM_512, F_fcn_filtr_abs[:,:,0], rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)



F_lena_filtr = F_fcn_lena * F_fcn_filtr_abs

lena_ifft_2 = inverse_fourier(F_lena_filtr)
diff_2 = np.abs(I_lena-lena_ifft_2)

ploting_function(I_lena,lena_ifft_2,diff_2)